In [1]:
#setup for processed data
#Note: use for BinaryArray data produced from Entrainment_Preprocessing.ipynb
def GetProcessedString(PROCESSING=False):
    if PROCESSING==True:
        Processed_string="PROCESSED_"
    else:
        Processed_string=""
    return Processed_string

PROCESSING=False 
# PROCESSING=True #set to True if using Turbulence-Removed Binary Arrays
Processed_string = GetProcessedString(PROCESSING=PROCESSING)

In [2]:
####################################
#ENVIRONMENT SETUP

In [3]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [4]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [5]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [6]:
#IMPORT FUNCTIONS (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from FUNCTIONS_Variable_Calculation import *

In [7]:
####################################
#LOADING CLASSES

In [8]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="EntrainmentCalculation", dataName="EntrainmentCalculation",
                                dtype='int32')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 inputParcelDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Proje

In [19]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()

Running timesteps from 0:132 



In [10]:
####################################
#FUNCTIONS

In [11]:
def MakeDataDictionary(variableNames,t):
    timeString = ModelData.timeStrings[t]
    # print(f"Getting data from {timeString}","\n")
    
    dataDictionary = {variableName: CallLagrangianArray(ModelData, DataManager, timeString, variableName=variableName) 
                      for variableName in variableNames}      
    return dataDictionary
    
def GetSpatialData(t):    
    variableNames = ['Z', 'Y', 'X']
    dataDictionary = MakeDataDictionary(variableNames,t)
    Z,Y,X = (dataDictionary[k] for k in variableNames)
    return Z,Y,X

def GetAData(t):  
    variableNames = [f'{Processed_string}A_g', f'{Processed_string}A_c']
        
    dataDictionary = MakeDataDictionary(variableNames,t)
    A_g,A_c = (dataDictionary[k] for k in variableNames)
    return A_g,A_c

In [12]:
def SubtractA(A,A_Prior):
    D = np.zeros_like(A,dtype=np.int8)
    D = A*1 - A_Prior*1
    return D

In [13]:
# def TestTransferRatios(TransferEntrainment, TransferDetrainment,
#     # Print diagnostic means
#     print("TransferEntrainment (mean):", np.mean(TransferEntrainment))
#     print("TransferDetrainment (mean):", np.mean(TransferDetrainment))

#     # Compute vertical profiles of mean ratios (%)
#     ratio_e_profile = np.nanmean(TransferEntrainment_Ratio, axis=(1, 2)) * 100
#     ratio_d_profile = np.nanmean(TransferDetrainment_Ratio, axis=(1, 2)) * 100

#     # --- Plot ---
#     plt.figure(figsize=(6, 8))
#     plt.plot(ratio_e_profile, zh, label='Transfer Entrainment', color='blue')
#     plt.plot(ratio_d_profile, zh, label='Transfer Detrainment', color='red')
#     plt.xlabel('Transfer Ratio (%)')
#     plt.ylabel('z (km)')
#     plt.title(plot_title)
#     plt.legend()
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()

#     return ratio_e_profile, ratio_d_profile

In [14]:
def CalculateEntrainment(t, Z,Y,X,Z_Prior,Y_Prior,X_Prior, A1,A2, A1_Prior,A2_Prior):
    """
    Function to compute 3D entrainment and update result array based on provided inputs.
    
    Returns a 3D (t,z) array containing the sum of the D array representing entrained parcels, by 1, and detrained parcels, by -1.
    The finally array is then ordered by the appropiate index using the np.add.at function
    
    Parameters:
    - A: The (t,p) lagrangian binary array.
    - T: The (t,p) lagrangian time index array.
    - Z: The (t,p) Lagrangian z index array.
    - Y: The (t,p) Lagrangian y index array.
    - X: The (t,p) Lagrangian x index array.

    """
    #Calculation for Entrainment and Detrainment
    DMatrix_Entrainment = SubtractA(A2,A2_Prior)
    DMatrix_Detrainment = DMatrix_Entrainment.copy()

    # Update D for entrainment/detrainment
    DMatrix_Entrainment[DMatrix_Entrainment < 0] = 0
    DMatrix_Detrainment[DMatrix_Detrainment > 0] = 0

    ##########
    #General <==> Cloudy Updraft-Transfer Entrainment
    ########## c to g AND g to c
    SMatrix_Entrainment = ((A1_Prior == 1) & (A2 == 1) & (A2_Prior == 0)).astype(np.int8) #A1==>A2 entrainment
    SMatrix_Detrainment = SMatrix_Entrainment.copy()
    #last conditional removes double-counting where A1 = A2 = 1)
    ##########
    
    
    # Initialize time and vertical dimension arrays
    Nz = ModelData.Nzh; Ny = ModelData.Nyh; Nx = ModelData.Nxh
    
    # Initialize result array
    Entrainment = np.zeros((Nz, Ny, Nx),dtype="int32")
    Detrainment = Entrainment.copy()
    TransferEntrainment = Entrainment.copy()
    TransferDetrainment = Entrainment.copy()

    if t==0:
        return Entrainment,Detrainment, TransferEntrainment,TransferDetrainment
    else:
        # Use np.add.at to accumulate values in the result array
        np.add.at(Entrainment, (Z, Y, X), DMatrix_Entrainment)
        np.add.at(Detrainment, (Z_Prior, Y_Prior, X_Prior), DMatrix_Detrainment)
    
        np.add.at(TransferEntrainment, (Z, Y, X), SMatrix_Entrainment)
        np.add.at(TransferDetrainment, (Z_Prior, Y_Prior, X_Prior), SMatrix_Detrainment) #*#*

        #TESTING (transfer entrainment)
        # ratio_e_profile, ratio_d_profile = TestTransferRatios(
        #     TransferEntrainment, TransferDetrainment,
        #     TransferEntrainment_Ratio, TransferDetrainment_Ratio,
        #     ModelData.zh
        # )

        return Entrainment,Detrainment, TransferEntrainment,TransferDetrainment

In [15]:
def RunCalculation(t, Z,Y,X,Z_Prior,Y_Prior,X_Prior, A_g,A_c, A_g_Prior,A_c_Prior, Processed_string): 

    [Entrainment_g, Detrainment_g,
     TransferEntrainment_g,TransferDetrainment_c] = CalculateEntrainment(t, Z,Y,X,Z_Prior,Y_Prior,X_Prior, A1=A_c,A2=A_g, A1_Prior=A_c_Prior,A2_Prior=A_g_Prior)

    [Entrainment_c, Detrainment_c,
     TransferEntrainment_c,TransferDetrainment_g] = CalculateEntrainment(t, Z,Y,X,Z_Prior,Y_Prior,X_Prior, A1=A_g,A2=A_c, A1_Prior=A_g_Prior,A2_Prior=A_c_Prior)

    outputDictionary_Entrainment = {
        f"{Processed_string}Entrainment_g": Entrainment_g,
        f"{Processed_string}Entrainment_c": Entrainment_c,
        f"{Processed_string}TransferEntrainment_g": TransferEntrainment_g, #c to g
        f"{Processed_string}TransferEntrainment_c": TransferEntrainment_c, #g to c
        }
    
    outputDictionary_Detrainment = {
        f"{Processed_string}Detrainment_g": Detrainment_g,
        f"{Processed_string}Detrainment_c": Detrainment_c,
        f"{Processed_string}TransferDetrainment_g": TransferDetrainment_g, #from g to c
        f"{Processed_string}TransferDetrainment_c": TransferDetrainment_c, #from c to g
        }

    return outputDictionary_Entrainment, outputDictionary_Detrainment

In [16]:
##############################################
#RUNNING

In [20]:
#running calculation
for t in num_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    #loading data
    [Z,Y,X] = GetSpatialData(t)
    [Z_Prior,Y_Prior,X_Prior] = GetSpatialData(t-1)
    [A_g,A_c] = GetAData(t)
    [A_g_Prior,A_c_Prior] = GetAData(t-1) 

    #calculating variables
    [outputDictionary_Entrainment,outputDictionary_Detrainment] = RunCalculation(t, Z,Y,X,Z_Prior,Y_Prior,X_Prior, A_g,A_c, A_g_Prior,A_c_Prior, Processed_string)
    
    #outputting
    timeString = ModelData.timeStrings[t]
    timeString_Prior = ModelData.timeStrings[t-1]
    
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary_Entrainment, dataName=f"{Processed_string}Entrainment")
    if t!=0:
        DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString_Prior, outputDictionary_Detrainment, dataName=f"{Processed_string}Detrainment")

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/EntrainmentCalculation/Entrainment_1km_5min_34nz_0-00-00.h5 

Current time 1
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/EntrainmentCalculation/Entrainment_1km_5min_34nz_0-05-00.h5 

Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/EntrainmentCalculation/Detrainment_1km_5min_34nz_0-00-00.h5 

Current time 2
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/EntrainmentCalculation/1km_5min_34nz/EntrainmentCalculation/Entrainment_1km_5min_34nz_0-10-00.h5 

Saved timestep 

In [ ]:
####################################
#CALCULATING ENTRAINMENT CONSTANT

In [ ]:
#constants
def GetConstants():
    Cp=1004 #Jkg-1K-1
    Cv=717 #Jkg-1K-1
    Rd=Cp-Cv #Jkg-1K-1
    eps=0.608
    return Cp,Cv,Rd,eps

def GetNumerics():
    Np=ModelData.Np #number of lagrangian parcles

    # Lx=(ModelData.xf[-1]-ModelData.xf[0])*1000 #x length (m)
    # Ly=(ModelData.yf[-1]-ModelData.yf[0])*1000 #y length (m)
    dt=(ModelData.time[1]-ModelData.time[0]).item()/1e9 #sec
    dy=(ModelData.yf[1]-ModelData.yf[0])*1e3 #meters
    dx=(ModelData.xf[1]-ModelData.xf[0])*1e3 #meters
    
    zfs=ModelData.zf*1000
    dz = np.diff(zfs)
    return Np, dt, dz,dy,dx

def zf(k):
    out=ModelData.zf[k]*1000
    return out

#calculation functions
# def rho(x,y,z,t):
#     p=data['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
#     p0=101325 #Pa
#     theta=data['th'].isel(xh=x,yh=y,zh=z,time=t).item()
#     T=theta*(p/p0)**(Rd/Cp)
#     qv=data['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
#     # Tv=T*(1+eps*qv)
#     Tv=T*(eps+qv)/(eps*(1+qv))
#     rho = p/(Rd*Tv)
#     out=rho
#     return out
    
def rho(x,y,z,rho_data_t):
    out=rho_data_t[z,y,x]
    return out
    
def Calculate_dm(t, dz,dy,dx, Np):
    timeString = ModelData.timeStrings[t]
    rho_data_t = CallVariable(ModelData, DataManager, timeString, "rho")
    
    #calculating
    m=0
    for k in range(ModelData.Nzh):
        dz=(zf(k+1)-zf(k))
        for j in range(ModelData.Nyh):
            for i in range(ModelData.Nxh):
                rho_out=rho(i,j,k,rho_data_t)
                m+=rho_out*dz

    #multiplying by integration differentials
    dm = m*dx*dy/Np
    return dm

In [ ]:
def ComputeEntrainmentConstant(t=0):
    #constants
    [Cp,Cv,Rd,eps] = GetConstants()
    [Np, dt, dz,dy,dx] = GetNumerics()

    #calculation
    dm = Calculate_dm(t, dz,dy,dx, Np)
    divisor=dt*dz*dy*dx
    entrainmentConstant = dm/divisor

    outputDictionary = {"entrainmentConstant": entrainmentConstant}
    return outputDictionary

In [ ]:
#calculating
outputDictionary_EntrainmentConstant = ComputeEntrainmentConstant()
entrainmentConstant = outputDictionary_EntrainmentConstant["entrainmentConstant"]

#saving
DataManager.SaveCalculations(DataManager.outputDataDirectory, outputDictionary_EntrainmentConstant, dataName="EntrainmentConstant",dtype="float32")

#plotting
plt.plot(entrainmentConstant,ModelData.zh,color='black')
plt.ylabel("z (km)")
plt.xlabel("Entrainment Constant (kg/m^3/s)")
plt.title("Plotting Vertical Profile of Entrainment Constant\n (due to stretched z-grid)");